In [0]:
%sql
-- Ad Revenue Query
WITH ad_revenue AS (
  SELECT 
    u.user_id,
    DATE_TRUNC('DAY', FROM_UNIXTIME(u.first_user_time)) AS aq_timeline,
    DATEDIFF(DAY, FROM_UNIXTIME(u.first_user_time), FROM_UNIXTIME(ue.time)) AS diff,
    SUM(ue.properties.revenue) AS ads_revenue
  FROM 
    workspace.dashtoon_data.new_user_profile_dlt u
  LEFT JOIN 
    workspace.dashtoon_data.mixpanel_events_dlt ue ON u.user_id = ue.user_id
  WHERE 
    ue.event_name = 'adPaid'
  GROUP BY 
    1, 2, 3
)

SELECT
  aq_timeline,
  ROUND(SUM(CASE WHEN diff = 0 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D0,
  ROUND(SUM(CASE WHEN diff = 1 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D1,
  ROUND(SUM(CASE WHEN diff = 2 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D2,
  ROUND(SUM(CASE WHEN diff = 3 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D3,
  ROUND(SUM(CASE WHEN diff = 4 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D4,
  ROUND(SUM(CASE WHEN diff = 5 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D5,
  ROUND(SUM(CASE WHEN diff = 6 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D6,
  ROUND(SUM(CASE WHEN diff = 7 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D7,
  ROUND(SUM(CASE WHEN diff = 14 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D14,
  ROUND(SUM(CASE WHEN diff = 28 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D28,
  ROUND(SUM(CASE WHEN diff = 60 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D60,
  ROUND(SUM(CASE WHEN diff = 90 THEN ads_revenue END)/COUNT(DISTINCT user_id), 2) AS D90
FROM 
  ad_revenue
GROUP BY 
  1
ORDER BY 
  1 DESC

In [0]:
%sql
-- Average Watch Time
WITH watch_time AS (
  SELECT 
    u.user_id,
    -- Option 1: Try using TO_DATE function
    TO_DATE(FROM_UNIXTIME(u.first_user_time)) AS aq_timeline,    
    DATEDIFF(DAY, FROM_UNIXTIME(u.first_user_time), FROM_UNIXTIME(ue.time)) AS diff,
    SUM(CAST(ue.properties.timeSpent AS INT)) AS timeSpent
  FROM 
     workspace.dashtoon_data.new_user_profile_dlt u
  LEFT JOIN 
   workspace.dashtoon_data.mixpanel_events_dlt ue ON u.user_id = ue.user_id
  WHERE 
    ue.event_name IN ('reelForegroundWatchTime')
    AND u.first_user_time IS NOT NULL -- Make sure to filter out NULL values
  GROUP BY 
    1, 2, 3
)

SELECT
  aq_timeline,
  ROUND(SUM(CASE WHEN diff = 0 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D0,
  ROUND(SUM(CASE WHEN diff = 1 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D1,
  ROUND(SUM(CASE WHEN diff = 2 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D2,
  ROUND(SUM(CASE WHEN diff = 3 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D3,
  ROUND(SUM(CASE WHEN diff = 4 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D4,
  ROUND(SUM(CASE WHEN diff = 5 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D5,
  ROUND(SUM(CASE WHEN diff = 6 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D6,
  ROUND(SUM(CASE WHEN diff = 7 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D7,
  ROUND(SUM(CASE WHEN diff = 14 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D14,
  ROUND(SUM(CASE WHEN diff = 28 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D28,
  ROUND(SUM(CASE WHEN diff = 60 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D60,
  ROUND(SUM(CASE WHEN diff = 90 THEN timeSpent END)/NULLIF(COUNT(DISTINCT user_id), 0), 2) AS D90
FROM 
  watch_time
GROUP BY 
  1
ORDER BY 
  1 DESC

In [0]:
%sql
--CrossWalk Shows

WITH show_2_activations AS (
  SELECT
    user_id,
    event_name,
    properties.showName,
    time
  FROM 
     workspace.dashtoon_data.mixpanel_events_dlt
),

ft AS (
  SELECT 
    u.first_activated_show AS show1,
    s2.showName AS show2,
    COUNT(DISTINCT s2.user_id) AS users
  FROM 
     workspace.dashtoon_data.new_user_profile_dlt u
  LEFT JOIN 
    show_2_activations s2 ON s2.user_id = u.user_id
  WHERE 
    s2.showName IS NOT NULL
    -- [[and u.country_code = {{country_code}}]]
    -- [[and u.os = {{os}}]]
  GROUP BY 
    1, 2
)

SELECT *
FROM (
  SELECT
    show1 AS first_show_activated,
    show2 AS top_cross_walk_shows,
    ROW_NUMBER() OVER (PARTITION BY show1 ORDER BY users DESC) AS rn
  FROM 
    ft
  WHERE 
    show1 IS NOT NULL
    
)
WHERE 
  rn <= 3

In [0]:
%sql
--OverAll Show Details
SELECT 
  ue.properties.showName,
  ue.properties.showId,
  COUNT(DISTINCT CASE WHEN ue.event_name = 'showOpen' THEN u.user_id END) AS showOpen_users,
  COUNT(DISTINCT CASE WHEN ue.event_name = 'reelOpen' THEN u.user_id END) AS any_reelOpen_users,
  COUNT(DISTINCT CASE WHEN ue.event_name = 'reelOpen' AND CAST(ue.properties.reelSequence AS INT) = 1 THEN u.user_id END) AS `1st_reelOpen_users`,
  COUNT(DISTINCT CASE WHEN ue.event_name LIKE 'show%Activated' THEN u.user_id END) AS activated_users,
  COUNT(DISTINCT CASE WHEN ue.event_name LIKE 'show%Activated' THEN u.user_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN ue.event_name = 'showOpen' THEN u.user_id END), 0) AS `activation%`,
  COUNT(DISTINCT CASE WHEN ue.event_name LIKE 'show%ActivatedLevel1' THEN u.user_id END) AS L1_activated_users,
  COUNT(DISTINCT CASE WHEN ue.event_name LIKE 'show%ActivatedLevel1' THEN u.user_id END) / 
    NULLIF(COUNT(DISTINCT CASE WHEN ue.event_name = 'showOpen' THEN u.user_id END), 0) AS `L1_activation%`,
  COUNT(DISTINCT CASE WHEN u.is_active_subscriber = 1 THEN u.user_id END) AS active_subscribers,
  SUM(CASE WHEN ue.event_name = 'adPaid' THEN ue.properties.revenue END) AS ads_revenue
FROM 
   workspace.dashtoon_data.new_user_profile_dlt u
LEFT JOIN 
   workspace.dashtoon_data.mixpanel_events_dlt ue ON u.user_id = ue.user_id
WHERE 
  TO_TIMESTAMP(u.first_user_time) >= TO_TIMESTAMP('2025-02-01')
  AND ue.properties.showId IS NOT NULL
  AND ue.properties.showName IS NOT NULL
GROUP BY 
  ue.properties.showName,
  ue.properties.showId
ORDER BY 
  ue.properties.showName